In [2]:
#encoding=utf-8
#语种检测
import jieba
in_f = open("/Users/hcnucai/Documents/KevinSwift/机器学习资料/02-自然语言处理班视频教程附讲义代码/课件资料/Lecture_2/Language-Detector/data.csv");
lines = in_f.readlines();
in_f.close();
dataset = [(line.strip()[:-3],line.strip()[-2:]) for line in lines];
#输出数据
print (dataset[:5]);

[('1 december wereld aids dag voorlichting in zuidafrika over bieten taboes en optimisme', 'nl'), ('1 millón de afectados ante las inundaciones en sri lanka unicef está distribuyendo ayuda de emergencia srilanka', 'es'), ('1 millón de fans en facebook antes del 14 de febrero y paty miki dani y berta se tiran en paracaídas qué harías tú porunmillondefans', 'es'), ('1 satellite galileo sottoposto ai test presso lesaestec nl galileo navigation space in inglese', 'it'), ('10 der welt sind bei', 'de')]


In [3]:
#把原始数据分为训练集和测试集

In [4]:
from sklearn.model_selection import train_test_split
#将对象打包成一个个的元组 利用*解压成列表
x,y = zip(*dataset); #反解压
#random_state：是随机数的种子。
x_train,x_test,y_train,y_test = train_test_split(x,y,random_state = 1);
print (len(x_train));

6799


In [5]:
#去掉噪声数据
import re
def remove_noise(document):
    noise_pattern = re.compile("|".join(["http\S+","\@\w+", "\#\w+"]));
    clean_text = re.sub(noise_pattern,"",document);
    return clean_text.strip();
remove_noise("Trump images are now more popular than cat gifs. @trump #trends http://www.trumptrends.html")

'Trump images are now more popular than cat gifs.'

In [13]:
#抽取降噪后数据上的特征 采用1-gram 和 2-gram #搜索关键词
#将文本数据转化为特征向量的过程 比较常用的有词袋法 CountVectorizer只考虑词汇在文本中出现的频率
from sklearn.feature_extraction.text import CountVectorizer
vec = CountVectorizer(
    lowercase = True,
    analyzer = 'char_wb', 
    ngram_range = (1,2),
    max_features = 1000, #最多的特征数
    preprocessor = remove_noise #降噪的
);
def get_features(x):
    vec.transform(x);

In [7]:
#把贝叶斯import 进来进行分类

In [8]:
from sklearn.naive_bayes import MultinomialNB;
classifier = MultinomialNB();
classifier.fit(vec.transform(x_train),y_train);

In [10]:
print (classifier.score(vec.transform(x_test),y_test));

0.9770621967357741


In [21]:
#规范化 写成一个class
import re;
from sklearn.feature_extraction.text import CountVectorizer;
from sklearn.model_selection import train_test_split
from sklearn.naive_bayes import MultinomialNB
class LanguageDetector():
   
    def __init__(self, classifier=MultinomialNB()):
        self.classifier = classifier
        self.vectorizer = CountVectorizer(ngram_range=(1,2), max_features=1000, preprocessor=self._remove_noise)

    def _remove_noise(self, document):
        noise_pattern = re.compile("|".join(["http\S+", "\@\w+", "\#\w+"]))
        clean_text = re.sub(noise_pattern, "", document)
        return clean_text

    def features(self, X):
        return self.vectorizer.transform(X)

    def fit(self, X, y):
        self.vectorizer.fit(X)
        self.classifier.fit(self.features(X), y)

    def predict(self, x):
        return self.classifier.predict(self.features([x]))

    def score(self, X, y):
        return self.classifier.score(self.features(X), y)
in_f = open("/Users/hcnucai/Documents/KevinSwift/机器学习资料/02-自然语言处理班视频教程附讲义代码/课件资料/Lecture_2/Language-Detector/data.csv");
lines = in_f.readlines()
in_f.close()
dataset = [(line.strip()[:-3], line.strip()[-2:]) for line in lines]
x, y = zip(*dataset)
x_train, x_test, y_train, y_test = train_test_split(x, y, random_state=1)

language_detector = LanguageDetector()
language_detector.fit(x_train, y_train)
print(language_detector.predict('algunas cuentas del f煤tbol europeo y'))
print(language_detector.score(x_test, y_test))

['it']
0.9770621967357741
